In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset


model_name = "mideind/IceBERT"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)
RANDOM_SEED = 42


def tokenize_data(data, tokenizer, max_len=512):
    return tokenizer(
        data.tolist(), padding="max_length", truncation=True, max_length=max_len)

df = pd.read_csv("../Google-without-lem.csv")

df = df.sample(n=10000, random_state=RANDOM_SEED)
df.drop(["Unnamed: 0"], axis=1, inplace=True)


def convert(sentiment):
    return 1 if sentiment == "positive" else 0


df["sentiment"] = df.sentiment.apply(convert)

# show how many positive and negative reviews we have
# print(df.sentiment.value_counts())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# df["review"] = df.review.apply(lambda x: x.replace("_NEG", ""))
torch.manual_seed(RANDOM_SEED)

X_train, X_temp, y_train, y_temp = train_test_split(
    df["review"], df["sentiment"], test_size=0.3, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

# Tokenize the training data
train_data = tokenize_data(X_train, tokenizer)

# Tokenize the validation data
val_data = tokenize_data(X_val, tokenizer)

# Tokenize the test data
test_data = tokenize_data(X_test, tokenizer)


c:\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at mideind/IceBERT and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:

import numpy as np

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])  # ensure labels are included
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = SentimentDataset(train_data, y_train)
val_dataset = SentimentDataset(val_data, y_val)
test_dataset = SentimentDataset(test_data, y_test)

# Set GPU


# class CustomTrainer(Trainer):
#     def log(self, logs: dict):
#         # Get the last evaluation loss.
#         eval_loss = self.evaluate()["eval_loss"]
#         logs["eval_loss"] = eval_loss

#         # Call the parent class log method to handle the rest.
#         super().log(logs)

# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="steps",  # "steps" to evaluate each `logging_steps` or "epoch" to evaluate each epoch
#     eval_steps=1000,  # Evaluation and Save happens every 500 steps
#     num_train_epochs=10,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir="./logs",
# )


def compute_metrics(p):
    return {"acc": (np.argmax(p.predictions, axis=1) == p.label_ids).mean()}

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=True,  # This will ensure that the best model is loaded at the end of training
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.01)],
)


In [3]:
trainer.train()


results = trainer.evaluate()
print(results)

model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")

  6%|▌         | 500/8750 [02:11<36:16,  3.79it/s]

{'loss': 0.5666, 'learning_rate': 5e-05, 'epoch': 0.57}


                                                  
  6%|▌         | 500/8750 [02:27<36:16,  3.79it/s]

{'eval_loss': 0.4655722379684448, 'eval_acc': 0.84, 'eval_runtime': 16.371, 'eval_samples_per_second': 91.626, 'eval_steps_per_second': 11.484, 'epoch': 0.57}


 11%|█▏        | 1000/8750 [04:39<34:14,  3.77it/s]  

...

 51%|█████▏    | 4500/8750 [22:39<21:23,  3.31it/s]


{'train_runtime': 1359.3772, 'train_samples_per_second': 51.494, 'train_steps_per_second': 6.437, 'train_loss': 0.31269213019477, 'epoch': 5.14}


100%|██████████| 188/188 [00:16<00:00, 11.49it/s]


{'eval_loss': 0.4011671841144562, 'eval_acc': 0.8853333333333333, 'eval_runtime': 16.3446, 'eval_samples_per_second': 91.773, 'eval_steps_per_second': 11.502, 'epoch': 5.14}


('./sentiment_model\\tokenizer_config.json',
 './sentiment_model\\special_tokens_map.json',
 './sentiment_model\\vocab.json',
 './sentiment_model\\merges.txt',
 './sentiment_model\\added_tokens.json',
 './sentiment_model\\tokenizer.json')